# GenreMatch
Data cleaning and model generation  
*Jeremy Freedman, Reza Madhavan, Kunal Sheth*

In [111]:
!unzip Archive.zip

unzip:  cannot find or open Archive.zip, Archive.zip.zip or Archive.zip.ZIP.


In [112]:
!ls utils

__pycache__  genius.py		grab_lyrics.py
azlyrics.py  grab_genius.ipynb	songlyrics.py


In [113]:
import pandas as pd
import numpy as np
import utils.songlyrics as sl
from collections import defaultdict
from nltk.corpus import stopwords as sw
from nltk import download as nltk_download
import sklearn as sk
from sklearn import tree
import collections
import gc
import math

In [114]:
a=pd.read_csv('data/song_lyrics.csv').dropna()
new_df = a.copy()
head = new_df.head(5)

In [115]:
# download the NLTK stopwords list, if necessary
nltk_download('stopwords')

[nltk_data] Downloading package stopwords to /home/jerem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [116]:
# we need to clean and update the NLTK stopwords for our data
# we're stripping out punctuation entirely, which the stopwords are not equipped to handle
# could optionally add music-centric stopwords ('oh', 'yeah', 'like', etc)

temp_words = sw.words('english')
stopwords = []
additions = ['im', 'ill', 'id', 'oh', 'cant', 'ive']
for w in temp_words:
    stopwords.append(sl._clean(w))
stopwords += additions
print(stopwords)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'youre', 'youve', 'youll', 'youd', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'shes', 'her', 'hers', 'herself', 'it', 'its', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'thatll', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', '

In [117]:
for index,row in new_df.iterrows():
  if index % 100 == 0:
    print(index)

  song = row['Lyrics']
  x=song.find('\n')
  song2 = sl._clean(song[x+1:])
  new_song = ''
  for i in range(len(song2)):
    char = song2[i]

    if char == '\n':
      new_song += ' '
    elif char == '(' or char == ')':
      new_song += ''
    else:
      new_song += char


    new_song = new_song.encode('ascii','ignore').decode()

  song2 = new_song.split(' ')
  song2 = list(filter(lambda x : x!='' and x not in stopwords, song2))
  song2 = song2[:len(song2)-1]

  new_df.at[index, 'Lyrics'] = song2


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000


In [118]:
wordfreq = []
word_appearance_cnt = defaultdict(int) # how many songs contain word W? used for IDF
for index,row in new_df.iterrows():
  lyrics = row['Lyrics']
  x = dict(collections.Counter(lyrics))
  for w in x:
    word_appearance_cnt[w] += 1
  # mapped = []
  # for k in x.keys():
  #   mapped.append((k,x[k]))
  wordfreq.append(x)

new_df['WordFreq'] = wordfreq

In [119]:
new_df['UniqueWords']= new_df.apply(lambda row : len(row['WordFreq']), axis = 1)
new_df

,Unnamed: 0,Artist,Song,Lyrics,Genre,WordFreq,UniqueWords
0,0,One Direction,Perfect,"[might, never, knight, shinin, armor, might, n...",pop,"{'might': 4, 'never': 4, 'knight': 1, 'shinin'...",71
1,1,One Direction,Story of My Life,"[written, walls, stories, explain, leave, hear...",pop,"{'written': 3, 'walls': 2, 'stories': 1, 'expl...",68
2,2,One Direction,Drag Me Down,"[got, fire, heart, scared, dark, never, seen, ...",pop,"{'got': 4, 'fire': 2, 'heart': 2, 'scared': 2,...",37
3,3,One Direction,What Makes You Beautiful,"[insecure, know, turning, heads, walk, door, n...",pop,"{'insecure': 1, 'know': 18, 'turning': 1, 'hea...",54
4,4,One Direction,Night Changes,"[going, tonight, changes, something, red, moth...",pop,"{'going': 2, 'tonight': 3, 'changes': 8, 'some...",65
...,...,...,...,...,...,...,...
4074,4074,Def Leppard,Action! Not Words,"[sick, tired, damn, tv, gonna, make, movie, wa...",metal,"{'sick': 1, 'tired': 1, 'damn': 1, 'tv': 1, 'g...",61
4075,4075,Def Leppard,Rock Rock (Till You Drop),"[ready, get, set, tear, place, apart, need, ti...",metal,"{'ready': 1, 'get': 2, 'set': 1, 'tear': 1, 'p...",74
4076,4076,Def Leppard,Slang,"[sitting, dark, getting, taken, said, somethin...",metal,"{'sitting': 1, 'dark': 1, 'getting': 1, 'taken...",41
4077,4077,Def Leppard,Die Hard the Hunter,"[lets, welcome, home, soldier, boy, far, away,...",metal,"{'lets': 2, 'welcome': 2, 'home': 2, 'soldier'...",65


In [120]:
uniq_words = set()
for x in new_df['WordFreq']:
  uniq_words = uniq_words.union(x.keys())
uniq_wordict = {k : 0 for k in uniq_words}

In [125]:
word_df = new_df[['Genre', 'WordFreq', 'Artist', 'UniqueWords']]

for index,row in word_df.iterrows():
  if index % 100 == 0:
    print(f'\r{100 * index // len(new_df)}%', end='')
  for k in row['WordFreq']:
    # uniq_wordict[k] = row['WordFreq'][k] # original (raw count)
    uniq_wordict[k] = row['WordFreq'][k] * math.log(len(new_df) / word_appearance_cnt[k]) # tf-idf

  word_df.at[index, 'WordFreq'] = uniq_wordict
  word_df.at[index, 'Artist'] = list(uniq_wordict.values()) + [row['UniqueWords']]
  uniq_wordict = {k : 0 for k in uniq_words}

word_df = word_df.replace({'Artist':'WordFreqList'})


0%
2%
4%
7%
9%
12%
14%
17%
19%
22%
24%
26%
29%
31%
34%
36%
39%
41%
44%
46%
49%
51%
53%
56%
58%
61%
63%
66%
68%
71%
73%
76%
78%
80%
83%
85%
88%


KeyboardInterrupt: 

In [126]:
# free up memory
gc.collect()

109

In [ ]:
word_df = word_df.rename(columns = {'Artist':'WordFreqList'})
print(word_df)

      Genre                                           WordFreq  \
0       pop  {'might': 8.870149554808703, 'never': 3.944954...   
1       pop  {'written': 14.83819836344378, 'walls': 7.2440...   
2       pop  {'got': 3.1037276581915667, 'fire': 4.87686244...   
3       pop  {'insecure': 5.1778677352052505, 'know': 9.906...   
4       pop  {'going': 4.476031840091433, 'tonight': 7.1593...   
...     ...                                                ...   
4074  metal  {'sick': 3.4613316872147832, 'tired': 3.269936...   
4075  metal  {'ready': 2.995241957290184, 'get': 1.69999781...   
4076  metal  {'sitting': 3.5859741324220598, 'dark': 2.8121...   
4077  metal  {'lets': 4.614017583065335, 'welcome': 8.43803...   
4078  metal  {'saturday': 81.37902425081008, 'feel': 4.0420...   

                                           WordFreqList  UniqueWords  
0     [8.870149554808703, 3.9449546355004306, 6.3674...           71  
1     [14.83819836344378, 7.244028137810513, 4.87937...          

In [ ]:
# make a 80/20 train/test split of lyric words tagged by origin genre
X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(list(word_df['WordFreqList']), list(word_df['Genre']), test_size=0.2)

print(f'Split ratio: {len(X_test) / (len(X_test) + len(X_train))}\nTest set: {len(X_test)}\nTrain set: {len(X_train)}')

Split ratio: 0.20009808729769496
Test set: 816
Train set: 3262


In [ ]:
X_train[1][-1]

68

In [ ]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train,Y_train)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3262,) + inhomogeneous part.

In [ ]:
preds = clf.predict(X_test)

In [ ]:
np.sum(preds == Y_test)/len(preds)

0.39215686274509803




---
# STOP

In [ ]:
# I'm opening the csv of lyrics from songlyrics.com, which is structured differently than the genius one.
# might need to restructure this code (or the genius csv) if we want to use that too
# the only difference is all the lyrics by each artist are combined into a single cell (as opposed to split by song)
df = pd.read_csv('data/all_sl.csv')
print(f'Imported {len(df)} lines')
genres = set(df['Genre'])
print(f'Identified {len(genres)} genres: {genres}')
lyrics = defaultdict(str)
for genre in genres:
    for artist in list(df[df['Genre'] == genre]['Lyrics']): # grab each row (artist) and select the lyrics cell
        lyrics[genre] += artist # combine each batch of lyrics into the respective genre in the dictionary
print(f'Extracted {len(lyrics)} genres')

Imported 166 lines
Identified 7 genres: {'soul_rb', 'metal', 'alternative', 'rock', 'country', 'pop', 'rap_hiphop'}
Extracted 7 genres


In [ ]:
frequency_tables = {}
words_uniq = {}
lines_uniq = {}
for (genre,lyric) in lyrics.items():
    frequency_tables[genre] = sl.words_freq(lyric, stopwords)
    words_uniq[genre] = sl.words(lyric, stopwords)
    print(f'[{genre}] Identified {len(words_uniq[genre])} unique words')

[soul_rb] Identified 4626 unique words
[metal] Identified 8963 unique words
[alternative] Identified 5750 unique words
[rock] Identified 6416 unique words
[country] Identified 6533 unique words
[pop] Identified 7269 unique words
[rap_hiphop] Identified 15104 unique words


In [ ]:
len(lyrics['alternative'])

703734

In [ ]:
# as an example, print the top 25 most common terms from each genre!
for genre in frequency_tables:
    print(f'### {genre} ###\n{sorted(frequency_tables[genre].items(), key=lambda x: x[1], reverse=True)[:25]}')

### soul_rb ###
[('love', 1492), ('baby', 1220), ('know', 971), ('yeah', 762), ('like', 627), ('got', 547), ('want', 477), ('come', 457), ('time', 439), ('one', 435), ('wanna', 435), ('go', 424), ('let', 395), ('make', 364), ('get', 350), ('say', 346), ('right', 310), ('way', 309), ('see', 307), ('girl', 306), ('cause', 300), ('day', 296), ('aint', 274), ('never', 273), ('good', 264)]
### metal ###
[('one', 573), ('like', 565), ('never', 543), ('know', 507), ('time', 504), ('love', 494), ('see', 492), ('get', 473), ('away', 452), ('life', 449), ('take', 439), ('yeah', 434), ('let', 380), ('way', 368), ('die', 348), ('got', 348), ('feel', 343), ('go', 341), ('world', 337), ('eyes', 321), ('come', 316), ('inside', 303), ('say', 279), ('man', 274), ('want', 272)]
### alternative ###
[('know', 838), ('love', 678), ('time', 652), ('like', 599), ('go', 596), ('got', 543), ('one', 525), ('never', 506), ('take', 457), ('back', 447), ('away', 445), ('say', 440), ('come', 432), ('get', 418), ('w

In [ ]:
# make a 80/20 train/test split of lyric words tagged by origin genre
X = []
Y = []
for (genre,words) in words_uniq.items():
    X += words
    Y += [genre] * len(words)
X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2)
print(f'Split size: {len(X_test) / (len(X_test) + len(X_train))}\nTest set: {len(X_test)}\nTrain set: {len(X_train)}')


Split size: 0.20001463566345293
Test set: 10933
Train set: 43728
